In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 设置请求头部信息，模拟浏览器请求
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
# 设置YachtWorld网站上的搜索条件
search_params = {
    'class': 'sail',
    # 'price': 'USD100000-500000',
    # 'length': '40-50ft'
}
base_url = 'https://www.yachtworld.com'

requests.get(base_url, 
    headers=headers)

<Response [200]>

In [2]:
# 使用 Concurrent.futures 模块中的 ThreadPoolExecutor 类来实现异步请求
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
# pages = 1
pages = 666
req_list = list(range(pages))
# sn = requests.Session()
def transform(i):
    temp = requests.get(f'https://www.yachtworld.com/boats-for-sale/condition-used/type-sail/?page={i+1}', headers=headers).text
    req_list[i] = BeautifulSoup(temp, 'html.parser')
with ThreadPoolExecutor(max_workers=64) as t:
    tasks = [t.submit(transform, i) for i in range(pages)]
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        pass

100%|██████████| 666/666 [00:22<00:00, 29.36it/s]


In [3]:
soups = req_list
all(map(lambda x: isinstance(x, BeautifulSoup), soups))

True

In [9]:
soups[:4]

[<!DOCTYPE html>
 
 <html lang="en">
 <head>
 <meta charset="utf-8"/>
 <meta content="minimum-scale=1, initial-scale=1, width=device-width, shrink-to-fit=no, maximum-scale=1" name="viewport"/>
 <link as="style" href="https://fonts.googleapis.com/css2?family=Open+Sans:wght@400;700&amp;family=Lato:wght@300;900&amp;display=swap" onload="this.onload=null;this.rel='stylesheet'" rel="preload"/>
 <link as="style" href="/react-static/font-style.css" onload="this.onload=null;this.rel='stylesheet'" rel="preload"/>
 <title data-react-helmet="true">Used Sailboats for sale - YachtWorld</title>
 <meta content="noindex" data-react-helmet="true" name="robots"/><meta content="Find used sailboats for sale in your area &amp; across the world on YachtWorld. Offering the best selection of boats to choose from." data-react-helmet="true" name="description"/>
 <link as="image" href="//images.yachtworld.com/resize/1/22/22/8672222_0_270120231713_0.jpg?w=376&amp;h=253&amp;t=1674868392" rel="preload"/><link as="i

In [24]:
# soups[0].select_one('div.listing-card-information')
soups[551].select_one('#root > div.search > div.flex.flex-row > div.search-right-col > div.pagination-and-results-container.search-display > div.listings-container').find('a')


<a data-reporting-click-listing-type="standard listing" data-reporting-click-product-id="8699151" data-reporting-page="552" data-reporting-rank="8266" href="https://www.yachtworld.com/yacht/2008-grand-soleil-45-8699151/" name="&amp;lpos=P1S1&amp;lid=SR_Detail_Photo" property="itemListElement" typeof="http://schema.org/ListItem"><meta content="https://www.yachtworld.com/yacht/2008-grand-soleil-45-8699151/" property="url"/><meta content="1" property="position"/><div class="row listing-card regular" id="8699151"><div class="listing-card-image-container"><meta content="//images.yachtworld.com/resize/1/91/51/8699151_20230220015400209_1_LARGE.jpg?w=243&amp;h=155&amp;t=1676886952" property="image"/><img alt="Grand Soleil 45" class="listing-card-image" height="155" src="//images.yachtworld.com/resize/1/91/51/8699151_20230220015400209_1_LARGE.jpg?w=243&amp;h=155&amp;t=1676886952" width="243"/><div class="banner IN_STOCK">In-Stock</div></div><div class="listing-card-information"><h2 class="listi

In [29]:
from typing import List
i = 0
def get_urls_from_search_page(soup:BeautifulSoup):
    global i
    i=i+1
    try:
    # a = soup.select('#root > div.search > div.flex.flex-row > div.search-right-col > div.pagination-and-results-container.search-display > div.listings-container')
        # a = soup.select_one('#root > div.search > div.flex.flex-row > div.search-right-col > div.pagination-and-results-container.search-display > div.listings-container')
        # # a = soup.select_one('#root > div.search > div.flex.flex-row')
        # links = [link.get('href') for link  in a.find_all(name='a')]
        links = soup.select("#root > div.search > div.flex.flex-row > div.search-right-col > div.pagination-and-results-container.search-display > div.listings-container > a")
        
    except Exception as e:
        print(i)
        raise e
    return links
links = sum(map(get_urls_from_search_page, soups), [])
len(links)

9975

In [35]:
links = list(map(lambda x:x['href'] , links))

In [36]:
all(map(lambda x: x.startswith('https://'), links)) # 合法性
# [i for i in range(len(links)) if x.startswith('https://')]
len(links) # 数量
# unique的数量
len(set(links))


9975

In [ ]:
with open('old_links.txt', 'w') as f:
    f.write('\n'.join(links))